Version: 02.14.2023

# Natural Language Processing Walkthrough

In this notebook, you will use Sagemaker's built-in machine learning model __LinearLearner__ to predict the __isPositive__ field of the review dataset.

Overall dataset schema:
* __reviewText:__ Text of the review
* __summary:__ Summary of the review
* __verified:__ Whether the purchase was verified (True or False)
* __time:__ UNIX timestamp for the review
* __log_votes:__ Logarithm-adjusted votes log(1+votes)
* __isPositive:__ Whether the review is positive or negative (1 or 0)

You will follow these steps:

1. [Read the dataset](#1.-Reading-the-dataset)
2. [Exploratory Data Analysis](#2.-Exploratory-Data-Analysis)
3. [Text Processing: Stop words removal and stemming](#3.-Text-Processing:-Stop-words-removal-and-stemming)
4. [Training - Validation - Test Split](#4.-Training,-Validation,-and-Test-Split)
5. [Data processing with Pipeline and ColumnTransform](#5.-Data-processing-with-Pipeline-and-ColumnTransform)
6. [Train a classifier with SageMaker build-in algorithm](#6.-Train-a-classifier-with-SageMaker-built-in-algorithm)
7. [Model evaluation](#7.-Model-Evaluation)
8. [Deploy the model to an endpoint](#8.-Deploy-the-model-to-an-endpoint)
9. [Test the enpoint](#9.-Test-the-endpoint)
10. [Clean up model artifacts](#10.-Clean-up-model-artifacts)

In [ ]:
#Upgrade dependencies
!pip install --upgrade pip
!pip install --upgrade scikit-learn
!pip install --upgrade sagemaker

## 1. Reading the dataset
([Go to top](#Natural-Language-Processing-Walkthrough))

You will use the __pandas__ library to read the dataset.

In [ ]:
import pandas as pd

df = pd.read_csv('../data/AMAZON-REVIEW-DATA-CLASSIFICATION.csv')

print('The shape of the dataset is:', df.shape)

Look at the first five rows in the dataset.

In [ ]:
df.head(5)

You can change the options in the notebook to display more of the text data.

In [ ]:
pd.options.display.max_rows
pd.set_option('display.max_colwidth', None)
df.head()

In [ ]:
print(df.loc[[580]])

In [ ]:
df.dtypes

## 2. Exploratory Data Analysis
([Go to top](#Natural-Language-Processing-Walkthrough))

Let's look at the target distribution for our datasets.

In [ ]:
df['isPositive'].value_counts()

The business problem is concerned about finding the negative reviews (0). Model tuning for linear learner defaults to finding positive values (1). You can make this easier by flipping the negative (0) and positive values (1). By doing this, you can tune the model easier.

In [ ]:
df = df.replace({0:1, 1:0})
df['isPositive'].value_counts()

Check the number of missing values:

In [ ]:
df.isna().sum()

There are missing values in the text fields.

## 3. Text Processing: Stop words removal and stemming
([Go to top](#Natural-Language-Processing-Walkthrough))

In this task, you will remove some of the stop words, and perform stemming on the text data.

In [ ]:
# Install the library and functions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

You will create the stop word removal and text cleaning processes below. NLTK library provides a list of common stop words. You will use the list, but remove some of the words from that list. It is because those words are actually useful to understand the sentiment in the sentence.

In [ ]:
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

# Let's get a list of stop words from the NLTK library
stop = stopwords.words('english')

# These words are important for our problem. We don't want to remove them.
excluding = ['against', 'not', 'don', 'don\'t','ain', 'are', 'aren\'t', 'could', 'couldn\'t',
             'did', 'didn\'t', 'does', 'doesn\'t', 'had', 'hadn\'t', 'has', 'hasn\'t', 
             'have', 'haven\'t', 'is', 'isn\'t', 'might', 'mightn\'t', 'must', 'mustn\'t',
             'need', 'needn\'t','should', 'shouldn\'t', 'was', 'wasn\'t', 'were', 
             'weren\'t', 'won\'t', 'would', 'wouldn\'t']

# New stop word list
stop_words = [word for word in stop if word not in excluding]

snow = SnowballStemmer('english')

def process_text(texts): 
    final_text_list=[]
    for sent in texts:
        
        # Check if the sentence is a missing value
        if isinstance(sent, str) == False:
            sent = ''
            
        filtered_sentence=[]
        
        sent = sent.lower() # Lowercase 
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:
        
        for w in word_tokenize(sent):
            # We are applying some custom filtering here, feel free to try different things
            # Check if it is not numeric and its length>2 and not in stop words
            if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words):  
                # Stem and add to filtered list
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence) #final string of cleaned words
 
        final_text_list.append(final_string)
        
    return final_text_list

## 4. Training, Validation, and Test Split
([Go to top](#Natural-Language-Processing-Walkthrough))

Split the dataset into training (80%), validation (10%) and test (10%) using sklearn's [__train_test_split()__](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df[['reviewText', 'summary', 'time', 'log_votes']],
                                                  df['isPositive'],
                                                  test_size=0.20,
                                                  shuffle=True,
                                                  random_state=324
                                                 )

X_val, X_test, y_val, y_test = train_test_split(X_val,
                                                y_val,
                                                test_size=0.5,
                                                shuffle=True,
                                                random_state=324)

In [ ]:
print('Processing the reviewText fields')
X_train['reviewText'] = process_text(X_train['reviewText'].tolist())
X_val['reviewText'] = process_text(X_val['reviewText'].tolist())
X_test['reviewText'] = process_text(X_test['reviewText'].tolist())

print('Processing the summary fields')
X_train['summary'] = process_text(X_train['summary'].tolist())
X_val['summary'] = process_text(X_val['summary'].tolist())
X_test['summary'] = process_text(X_test['summary'].tolist())

The process_text() method in section 3 uses empty string for missing values.

## 5. Data processing with Pipeline and ColumnTransform
([Go to top](#Natural-Language-Processing-Walkthrough))

In the previous examples, we have seen how to use pipeline to prepare a data field for our machine learning model. This time, we will focus on multiple fields: numeric and text fields. 

   * For the numerical features pipeline, the __numerical_processor__ below, we use a MinMaxScaler (don't have to scale features when using Decision Trees, but it's a good idea to see how to use more data transforms). If different processing is desired for different numerical features, different pipelines should be built - just like shown below for the two text features.
   * For the text features pipeline, the __text_processor__ below, we use CountVectorizer() for the text fields.
   
The selective preparations of the dataset features are then put together into a collective ColumnTransformer, to be finally used in a Pipeline along with an estimator. This ensures that the transforms are performed automatically on the raw data when fitting the model and when making predictions, such as when evaluating the model on a validation dataset via cross-validation or making predictions on a test dataset in the future.

In [ ]:
# Grab model features/inputs and target/output
numerical_features = ['time',
                      'log_votes']

text_features = ['summary',
                 'reviewText']

model_features = numerical_features + text_features
model_target = 'isPositive'

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

### COLUMN_TRANSFORMER ###
##########################

# Preprocess the numerical features
numerical_processor = Pipeline([
    ('num_imputer', SimpleImputer(strategy='mean')),
    ('num_scaler', MinMaxScaler()) 
                                ])
# Preprocess 1st text feature
text_processor_0 = Pipeline([
    ('text_vect_0', CountVectorizer(binary=True, max_features=50))
                                ])

# Preprocess 2nd text feature (larger vocabulary)
text_precessor_1 = Pipeline([
    ('text_vect_1', CountVectorizer(binary=True, max_features=150))
                                ])

# Combine all data preprocessors from above (add more, if you choose to define more!)
# For each processor/step specify: a name, the actual process, and finally the features to be processed
data_preprocessor = ColumnTransformer([
    ('numerical_pre', numerical_processor, numerical_features),
    ('text_pre_0', text_processor_0, text_features[0]),
    ('text_pre_1', text_precessor_1, text_features[1])
                                    ]) 

### DATA PREPROCESSING ###
##########################

print('Datasets shapes before processing: ', X_train.shape, X_val.shape, X_test.shape)

X_train = data_preprocessor.fit_transform(X_train).toarray()
X_val = data_preprocessor.transform(X_val).toarray()
X_test = data_preprocessor.transform(X_test).toarray()

print('Datasets shapes after processing: ', X_train.shape, X_val.shape, X_test.shape)

## 6. Train a classifier with SageMaker built in algorithm
([Go to top](#Natural-Language-Processing-Walkthrough))

We will call the Sagemaker `LinearLearner()` below. 
* __Compute power:__ We will use `train_instance_count` and `train_instance_type` parameters. This example uses `ml.m4.xlarge` resource for training. We can change the instance type for our needs (For example GPUs for neural networks). 
* __Model type:__ `predictor_type` is set to __`binary_classifier`__, as we have a binary classification problem here; __`multiclass_classifier`__ could be used if there are 3 or more classes involved, or __'regressor'__ for a regression problem.

In [ ]:
import sagemaker

# Call the LinearLearner estimator object
linear_classifier = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                           instance_count=1,
                                           instance_type='ml.m4.xlarge',
                                           predictor_type='binary_classifier')

We are using the `record_set()` function of our binary_estimator to set the training, validation, test parts of the estimator. 

In [ ]:
train_records = linear_classifier.record_set(X_train.astype('float32'),
                                            y_train.values.astype('float32'),
                                            channel='train')
val_records = linear_classifier.record_set(X_val.astype('float32'),
                                          y_val.values.astype('float32'),
                                          channel='validation')
test_records = linear_classifier.record_set(X_test.astype('float32'),
                                           y_test.values.astype('float32'),
                                           channel='test')

`fit()` function applies a distributed version of the Stochastic Gradient Descent (SGD) algorithm and we are sending the data to it. We disabled logs with `logs=False`. You can remove that parameter to see more details about the process. __This process takes about 3-4 minutes on a ml.m4.xlarge instance.__

In [ ]:
linear_classifier.fit([train_records,
                       val_records,
                       test_records],
                       logs=False)

## 7. Model Evaluation
([Go to top](#Natural-Language-Processing-Walkthrough))

We can use Sagemaker analytics to get some performance metrics of our choice on the test set. This doesn't require us to deploy our model. Since this is a binary classfication problem, we can check the accuracy.

In [ ]:
sagemaker.analytics.TrainingJobAnalytics(linear_classifier._current_job_name, 
                                         metric_names = ['test:binary_classification_accuracy']
                                        ).dataframe()

## 8. Deploy the model to an endpoint
([Go to top](#Natural-Language-Processing-Walkthrough))

In the last part of this exercise, we will deploy our model to another instance of our choice. This will allow us to use this model in production environment. Deployed endpoints can be used with other AWS Services such as Lambda and API Gateway. A nice walkthrough is available here: https://aws.amazon.com/blogs/machine-learning/call-an-amazon-sagemaker-model-endpoint-using-amazon-api-gateway-and-aws-lambda/ if you are interested.

Run the following cell to deploy the model. We can use different instance types such as: `ml.t2.medium`, `ml.c4.xlarge` etc. __This will take some time to complete (Approximately 7-8 minutes).__

In [ ]:

linear_classifier_predictor = linear_classifier.deploy(initial_instance_count = 1,
                                                       instance_type = 'ml.t2.medium'
                                                      )

## 9. Test the endpoint
([Go to top](#Natural-Language-Processing-Walkthrough))

Let's use the deployed endpoint. We will send our test data and get predictions of it.

In [ ]:
import numpy as np

# Let's get test data in batch size of 25 and make predictions.
prediction_batches = [linear_classifier_predictor.predict(batch)
                      for batch in np.array_split(X_test.astype('float32'), 25)
                     ]

# Let's get a list of predictions
print([pred.label['score'].float32_tensor.values[0] for pred in prediction_batches[0]])

## 10. Clean up model artifacts
([Go to top](#Natural-Language-Processing-Walkthrough))

You can run the following to delete the endpoint after you are done using it.

In [ ]:
linear_classifier_predictor.delete_endpoint()

*©2023 Amazon Web Services, Inc. or its affiliates. All rights reserved. This work may not be reproduced or redistributed, in whole or in part, without prior written permission from Amazon Web Services, Inc. Commercial copying, lending, or selling is prohibited. All trademarks are the property of their owners.*
